# Convert Bike Count point Coordinates from GDA 94 to WGS 84
Data source uses the mapping projection system GDA 94 (A low distortion projection for the Melbourne area). 
Most web based maps, including google maps use web mercator WGS 84 (a projection used by many GPS based systems).   

This script converts the lat/lon point data in SuperTueDataCleaner/script_output/count_locations/count_location_details.csv 
to a digital web mercator for later mapping.

It also flags sites with bad coordinate data for further investigation

In [256]:
sourcefile = './script_output/count_locations/count_location_details.csv'

In [257]:
import pandas as pd

In [258]:
df = pd.read_csv(sourcefile, sep=', ', header = 0, engine = 'python')
df

,countsite,site_description,suburb,dist_from_cbd,easting,northing,melway_ref,primary_road,secondary_road
0,BW-CityLinkBrunswickRd,Brunswick Rd / Ormond Rd and Gibson St - Near ...,Brunswick West,5.7,318495.19,5817202.69,29 B6,Moonee Ponds Creek Trail,Brunswick Rd
1,BW-CityLinkDawsonSt,Dawson St and Jewel Crs - near CityLink,Brunswick West,7.1,318424.79,5817844.06,29 B8,Moonee Ponds Creek Trail,Dawson St
2,BW-CityLinkVictoriaSt,Victoria St (CityLink Overpass),Brunswick West,7.8,318346.84,5818350.13,29 E10,Moonee Ponds Creek Trail,Victoria St
3,BW-HopetounAvAlbionSt,Hopetoun Av Waxman Pde and Albion St,Brunswick West,8.8,318042.84,5819089.27,29 A4,Moonee Ponds Creek Trail,Albion St
4,BW-MelvilleDawsonSt,Melville St and Dawson St,Brunswick West,6.6,318908.20,5817775.38,29 C8,Melville Rd,Glenlyon - Dawson
5,BW-DalyStDawsonSt,Daly St South Daly St and Dawson St,Brunswick West,6.6,318907.57,5817774.70,29 C8,Daly St,Glenlyon - Dawson
6,BW-FlemingStParkSt,Fleming St and Park St,Brunswick West,5.6,319031.96,5817025.18,29 D10,Fleming St,Park St
7,BW-55TramParkSt,55 Tram and Park St,Brunswick West,5.2,319428.72,5816965.79,29 E10,West Brunswick Shimmy,Park St
8,BW-GranthamStBrunswickRd,Grantham St and Brunswick Rd,Brunswick,5.5,319352.37,5817091.00,29 D9,West Brunswick Shimmy,Brunswick Rd
9,BW-MelvilleRdVictoriaSt,Melville Rd and Victoria St,Brunswick West,7.2,318883.04,5818287.17,29 C6,Melville Rd,Victoria St


In [259]:
# GeoJSON will fail if there are no lat long coordinates,  drop these rows from the dataframe.
missingdata = df[df['easting'].isnull()]
missingdata

,countsite,site_description,suburb,dist_from_cbd,easting,northing,melway_ref,primary_road,secondary_road
34,BE-CanningBrunswick,Canning Street x Brunswick Rd Brunswick East,Brunswick East,4.9,NaN,NaN,30 A10,East Brunswick Shimmy,Brunswick Road


## Use pyproj to convert point coordinates

In [260]:
df = df.drop(missingdata.index)

In [261]:
import pyproj


def gda94_to_wgs84(series):
    '''
    Takes a series (a row of df), looks for 'easting' and 'northing' values
    Series must contain 'easting' and 'northing' values to work
    Creates an tuple 'lat lon' 
    
    '''
    easting = series['easting']
    northing = series['northing']
    
    gda94 = pyproj.Proj(init='epsg:28355')
    wgs84 = pyproj.Proj(init='epsg:4326')
    longitude, latitude = pyproj.transform(gda94, wgs84, easting, northing)

    latitude = round(latitude, 4)
    longitude = round(longitude, 4)
    series['latitude'] = latitude
    series['longitude'] = longitude
    return series


    

In [262]:
sitepoints = df.apply(gda94_to_wgs84,axis='columns')
sitepoints

,countsite,site_description,suburb,dist_from_cbd,easting,northing,melway_ref,primary_road,secondary_road,latitude,longitude
0,BW-CityLinkBrunswickRd,Brunswick Rd / Ormond Rd and Gibson St - Near ...,Brunswick West,5.7,318495.19,5817202.69,29 B6,Moonee Ponds Creek Trail,Brunswick Rd,-37.7745,144.9391
1,BW-CityLinkDawsonSt,Dawson St and Jewel Crs - near CityLink,Brunswick West,7.1,318424.79,5817844.06,29 B8,Moonee Ponds Creek Trail,Dawson St,-37.7687,144.9385
2,BW-CityLinkVictoriaSt,Victoria St (CityLink Overpass),Brunswick West,7.8,318346.84,5818350.13,29 E10,Moonee Ponds Creek Trail,Victoria St,-37.7641,144.9377
3,BW-HopetounAvAlbionSt,Hopetoun Av Waxman Pde and Albion St,Brunswick West,8.8,318042.84,5819089.27,29 A4,Moonee Ponds Creek Trail,Albion St,-37.7574,144.9344
4,BW-MelvilleDawsonSt,Melville St and Dawson St,Brunswick West,6.6,318908.20,5817775.38,29 C8,Melville Rd,Glenlyon - Dawson,-37.7694,144.9439
5,BW-DalyStDawsonSt,Daly St South Daly St and Dawson St,Brunswick West,6.6,318907.57,5817774.70,29 C8,Daly St,Glenlyon - Dawson,-37.7694,144.9439
6,BW-FlemingStParkSt,Fleming St and Park St,Brunswick West,5.6,319031.96,5817025.18,29 D10,Fleming St,Park St,-37.7762,144.9451
7,BW-55TramParkSt,55 Tram and Park St,Brunswick West,5.2,319428.72,5816965.79,29 E10,West Brunswick Shimmy,Park St,-37.7768,144.9496
8,BW-GranthamStBrunswickRd,Grantham St and Brunswick Rd,Brunswick,5.5,319352.37,5817091.00,29 D9,West Brunswick Shimmy,Brunswick Rd,-37.7757,144.9488
9,BW-MelvilleRdVictoriaSt,Melville Rd and Victoria St,Brunswick West,7.2,318883.04,5818287.17,29 C6,Melville Rd,Victoria St,-37.7648,144.9438


In [263]:
# Find and remove junk Lat Lon values.
# Any latitude further south than Wilsons Promontory 38.9333° S is junk

waytoosouth = sitepoints.query('latitude < -39')
waytoosouth 

,countsite,site_description,suburb,dist_from_cbd,easting,northing,melway_ref,primary_road,secondary_road,latitude,longitude
27,B-BarrowStAlbionSt,Barrow St and Albion St,Brunswick,6.9,321143.92,581869.75,29 J6,Barrow St,Albion St,-84.5657,129.8329
68,PV-LandellsRdKentRd,Kent Rd and Landells Rd,Pascoe Vale,12.9,318797.82,582300.48,17 C6,Landells Rd,Kent Rd,-84.5558,129.6332
69,PV-WestStBoundaryRd,West St Lake St and Pascoe St Boundary Rd),Pascoe Vale,13.4,318137.87,582363.37,17 B5,West St,Boundary Rd,-84.5535,129.5755


In [264]:
sitepoints= sitepoints.drop(waytoosouth.index)

## Convert to GeoJSON for use in mapping tools

###  Original Code developed by Geoff Boeing http://geoffboeing.com/2015/10/exporting-python-data-geojson/

In [265]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [266]:
cols = ['countsite', 'site_description', 'site_description', 'dist_from_cbd','primary_road','secondary_road']
geojson = df_to_geojson(sitepoints, cols)

In [267]:
import json

In [268]:
output_filename = './script_output/count_locations/count_location_details.json'
with open(output_filename, 'wb') as output_file:
    json.dump(geojson, output_file, indent=2) 